In [1]:
import requests                   
from bs4 import BeautifulSoup     
import pandas as pd               
import time                       
import numpy as np                
from functools import reduce      
import re    

In [2]:
# set year to scrape
year = 2020

# open csv
df = pd.read_csv('nfl_combine_2000_2020.csv')

# filter by year
df=dt[df['Year']==year]

# drop duplicates
url_list=df['NCAA_Link'].dropna().unique().tolist()

print(len(url_list))

FileNotFoundError: [Errno 2] No such file or directory: 'nfl_combine_2000_2020.csv'

In [3]:
def pullTable(url):
    tableID = ['defense','rushing','receiving','passing','punt_ret','kick_ret','kicking','punting']
    delays = [5, 6, 7, 8, 9, 10]

    headers = {
        'accept': '*/*',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9',
        'referer': 'http://www.google.com/',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
    }
       
    delay = np.random.choice(delays)
    time.sleep(delay)
    res = requests.get(url, headers = headers)
    ## Work around comments
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    tables = soup.findAll('table', id = tableID)
    table_list = pd.read_html(str(tables))

    # iterate over tables
    for table in table_list:
        headers=table.columns.tolist()
        if headers[6][0] == 'Receiving':
            table.columns = table.columns.droplevel(0) # drop multilevel columns
            table.columns = ['Year','School','Conf','Class','Pos','G','Rec','Rec_Yds','Rec_Avg','Rec_TD','Rush_Att','Rush_Yds','Rush_Avg','Rush_TD','Scrim_Plays','Scrim_Yds','Scrim_Avg','Scrim_TD']
        elif headers[6][0] == 'Rushing':
            table.columns = table.columns.droplevel(0)
            table.columns = ['Year','School','Conf','Class','Pos','G','Rush_Att','Rush_Yds','Rush_Avg','Rush_TD','Rec','Rec_Yds','Rec_Avg','Rec_TD','Scrim_Plays','Scrim_Yds','Scrim_Avg','Scrim_TD']
        elif headers[6][0] == 'Passing':
            table.columns = table.columns.droplevel(0)
            table.columns = ['Year','School','Conf','Class','Pos','G','Pass_Cmp','Pass_Att','Pass_Pct','Pass_Yds','Pass_Y/A','Pass_AY/A','Pass_TD','Pass_Int','Pass_Rate']
        elif headers[6][0] == 'Punt Ret':
            table.columns = table.columns.droplevel(0)
            table.columns = ['Year','School','Conf','Class','Pos','G','Punt_Ret','Punt_ret_Yds','Punt_ret_Avg','Punt_ret_TD','Kick_Ret','Kick_ret_Yds','Kick_ret_Avg','Kick_ret_TD']
        elif headers[6][0] == 'Kick Ret':
            table.columns = table.columns.droplevel(0)
            table.columns = ['Year','School','Conf','Class','Pos','G','Kick_Ret','Kick_ret_Yds','Kick_ret_Avg','Kick_ret_TD','Punt_Ret','Punt_ret_Yds','Punt_ret_Avg','Punt_ret_TD']
        elif headers[6][0] == 'Tackles':
            table.columns = table.columns.droplevel(0)
            table.columns = ['Year','School','Conf','Class','Pos','G','Def_Solo_Tackles','Def_Ast_Tackles','Def_Tot_Tackles','Def_Loss_Tackles','Def_Sk','Def_Int','Def_Int_Yds','Def_Int_Avg','Def_Int_TD','Def_Int_PD','Def_FR','Def_Yds_Fumbles','Def_TD_Fumbles','Def_FF']
        elif headers[6][0] == 'Kicking':
            table.columns = table.columns.droplevel(0)
            table.columns = ['Year','School','Conf','Class','Pos','G','Kick_XPM','Kick_XPA','Kick_XP_pct','Kick_FGM','Kick_FGA','Kick_FG_pct','Kick_Pts','Punts','Punt_Yds','Punt_Avg']
        elif headers[6][0] == 'Punting':
            table.columns = table.columns.droplevel(0)
            table.columns = ['Year','School','Conf','Class','Pos','G','Punts','Punt_Yds','Punt_Avg','Kick_XPM','Kick_XPA','Kick_XP_pct','Kick_FGM','Kick_FGA','Kick_FG_pct','Kick_Pts']
    
    # merge tables into one dataframe
    final = reduce(lambda x, y: pd.merge(x, y, on = ['Year','School','Conf','Class','Pos','G']), table_list)
    # insert ncaa link
    final.insert(0, 'NCAA_Link', url)
    
    return(final)

In [4]:
dfs = []
error_list = []

# iterate over list of urls
for url in url_list:
    try:
        table_data = pullTable(url)
        dfs.append(table_data)
    except Exception as e:
        # store the url and the error it causes in a list
        error = [url, e] 
        # append it to the list of errors
        error_list.append(error) 

# print errors
print(error_list)
# merge data into a single dataframe
combines_df = pd.concat(dfs, ignore_index=False, sort=False)
# save as csv
combines_df.to_csv('ncaa_player_ind_stats_draft.csv', index=False)

NameError: name 'url_list' is not defined

In [5]:
dfs = []
df_2000 = pd.read_csv('ncaa_player_ind_stats_2000.csv')
df_2001 = pd.read_csv('ncaa_player_ind_stats_2001.csv')
df_2002 = pd.read_csv('ncaa_player_ind_stats_2002.csv')
df_2003 = pd.read_csv('ncaa_player_ind_stats_2003.csv')
df_2004 = pd.read_csv('ncaa_player_ind_stats_2004.csv')
df_2005 = pd.read_csv('ncaa_player_ind_stats_2005.csv')
df_2006 = pd.read_csv('ncaa_player_ind_stats_2006.csv')
df_2007 = pd.read_csv('ncaa_player_ind_stats_2007.csv')
df_2008 = pd.read_csv('ncaa_player_ind_stats_2008.csv')
df_2009 = pd.read_csv('ncaa_player_ind_stats_2009.csv')
df_2010 = pd.read_csv('ncaa_player_ind_stats_2010.csv')
df_2011 = pd.read_csv('ncaa_player_ind_stats_2011.csv')
df_2012 = pd.read_csv('ncaa_player_ind_stats_2012.csv')
df_2013 = pd.read_csv('ncaa_player_ind_stats_2013.csv')
df_2014 = pd.read_csv('ncaa_player_ind_stats_2014.csv')
df_2015 = pd.read_csv('ncaa_player_ind_stats_2015.csv')
df_2016 = pd.read_csv('ncaa_player_ind_stats_2016.csv')
df_2017 = pd.read_csv('ncaa_player_ind_stats_2017.csv')
df_2018 = pd.read_csv('ncaa_player_ind_stats_2018.csv')
df_2019 = pd.read_csv('ncaa_player_ind_stats_2019.csv')
df_2020 = pd.read_csv('ncaa_player_ind_stats_2020.csv')
df_leftover = pd.read_csv('ncaa_player_ind_stats_cnc_error.csv')
df_draft_leftover = pd.read_csv('ncaa_player_ind_stats_draft_leftover.csv')
df_draft_leftover_2 = pd.read_csv('ncaa_player_ind_stats_draft_leftover_2.csv') 


dfs.append(df_2001)
dfs.append(df_2002)
dfs.append(df_2003)
dfs.append(df_2004)
dfs.append(df_2005)
dfs.append(df_2006)
dfs.append(df_2007)
dfs.append(df_2008)
dfs.append(df_2009)
dfs.append(df_2010)
dfs.append(df_2011)
dfs.append(df_2012)
dfs.append(df_2013)
dfs.append(df_2014)
dfs.append(df_2015)
dfs.append(df_2016)
dfs.append(df_2017)
dfs.append(df_2018)
dfs.append(df_2019)
dfs.append(df_2020)
dfs.append(df_leftover)
dfs.append(df_draft_leftover)
dfs.append(df_draft_leftover_2)

combines_df = pd.concat(dfs, ignore_index=False, sort=False)
combines_df.to_csv('ncaa_all_player_ind_stats_extended_v2.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'ncaa_player_ind_stats_2000.csv'